In [1]:
!pip install llama_index
from pathlib import Path
!pip install sentence_transformers

In [2]:
from google.colab import userdata
import os
os.environ['GEMINI_API_KEY'] = userdata.get('gemini_api_key')

In [3]:
!pip install llama_index.embeddings.huggingface
!pip install llama-index-embeddings-instructor
from google.colab import userdata
import os
os.environ['LLAMA_CLOUD_API_KEY'] = userdata.get('LlamaParse')
LLAMA_CLOUD_API_KEY=os.environ['LLAMA_CLOUD_API_KEY']
openai_api_key=userdata.get('ChatGPT')

In [4]:
import nest_asyncio
nest_asyncio.apply()
!pip install llama-index-core llama-parse llama-index-readers-file python-dotenv

# bring in deps
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    result_type="markdown"  # "markdown" and "text" are available
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=['/content/Final Policy document_LICs New Jeevan Shanti_V05_logo.pdf'], file_extractor=file_extractor).load_data()
print(documents)

Started parsing the file under job_id 6aee68e8-0602-43df-89b9-6997078e7995
[Document(id_='ceb75cc1-9b77-49ef-937b-1051d1aa2ace', embedding=None, metadata={'file_path': '/content/Final Policy document_LICs New Jeevan Shanti_V05_logo.pdf', 'file_name': 'Final Policy document_LICs New Jeevan Shanti_V05_logo.pdf', 'file_type': 'application/pdf', 'file_size': 588928, 'creation_date': '2024-10-16', 'last_modified_date': '2024-10-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# LIFE INSURANCE CORPORATION OF INDIA\n\n# (Established by the Life Insurance Corporation Act, 1956)\n\n# Registration Number: 512\n\n# LIC’S NEW JEEVAN SHANTI (UIN: 512N338V05)\n\n# (A Non-Linked, Non-Participating, Individual, Single Premium, Deferred Annuity Plan)\n\n# PART – A\n

In [5]:
!pip install llama_index.llms.gemini
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini


In [6]:
!pip install llama-index-embeddings-gemini
from llama_index.embeddings.gemini import GeminiEmbedding

In [7]:
from llama_index.core.node_parser import TokenTextSplitter
token_text_splitter = TokenTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separator="\n",
    backup_separators = [" ", "!", "?"]
)



In [16]:
from llama_index.core import VectorStoreIndex
import shutil
#shutil.rmtree('./index_openai')

In [17]:
# CReate Vector Store with Gemini Embedding

GOOGLE_API_KEY=userdata.get('gemini_api_key')
model_name = "models/embedding-001"
embed_model_gemini = GeminiEmbedding(model_name=model_name, api_key=GOOGLE_API_KEY)
index_gemini = VectorStoreIndex.from_documents(documents, embed_model=embed_model_gemini, transformations=[token_text_splitter])
!mkdir index_gemini
index_gemini.storage_context.persist(persist_dir="./index_gemini")
vector_retriever_gemini=index_gemini.as_retriever(similarity_top_k=3)


mkdir: cannot create directory ‘index_gemini’: File exists


In [10]:
# Create Vector Store with OpenAI Embedding
!pip install llama_index.embeddings.openai
from llama_index.embeddings.openai import OpenAIEmbedding
embed_model_openai = OpenAIEmbedding(api_key=openai_api_key)
index_openai = VectorStoreIndex.from_documents(documents, embed_model=embed_model_openai, transformations=[token_text_splitter])
!mkdir index_openai
index_openai.storage_context.persist(persist_dir="./index_openai")
vector_retriever_openai=index_openai.as_retriever(similarity_top_k=3)

mkdir: cannot create directory ‘index_openai’: File exists


In [18]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
huggingfacehub=userdata.get('huggingfacehub')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingfacehub
embed_model_bgesmall = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index_bgesmall = VectorStoreIndex.from_documents(documents, embed_model=embed_model_bgesmall, transformations=[token_text_splitter])
!mkdir index_bgesmall
index_bgesmall.storage_context.persist(persist_dir="./index_bgesmall")
vector_retriever_bgesmall=index_bgesmall.as_retriever(similarity_top_k=3)

mkdir: cannot create directory ‘index_bgesmall’: File exists


In [19]:
!pip install llama-index-embeddings-jinaai
!pip install llama-index-llms-openai
!pip install Pillow
from llama_index.embeddings.jinaai import JinaEmbedding
jinaai_api_key = userdata.get('jina')
os.environ["JINAAI_API_KEY"] = jinaai_api_key
embed_model_jina = JinaEmbedding(api_key=jinaai_api_key,model="jina-embeddings-v3",task="retrieval.passage",)
index_jina = VectorStoreIndex.from_documents(documents, embed_model=embed_model_jina, transformations=[token_text_splitter])
!mkdir index_jina
index_jina.storage_context.persist(persist_dir="./index_jina")
vector_retriever_jina=index_jina.as_retriever(similarity_top_k=3)


mkdir: cannot create directory ‘index_jina’: File exists


In [13]:
from llama_index.core import get_response_synthesizer
from llama_index.llms.gemini import Gemini
llm_gemini=Gemini(api_key=userdata.get('gemini_api_key'),model_name="models/gemini-1.5-flash")
response_synthesizer_gemini = get_response_synthesizer(response_mode="accumulate", llm=llm_gemini)
query_engine_gemini = index_gemini.as_query_engine(llm=llm_gemini,response_synthesizer=response_synthesizer_gemini)
response_gemini = query_engine_gemini.query("What are all the definitions?")
print(response_gemini)

Response 1: The provided context does not contain any definitions. 

---------------------
Response 2: The definitions provided are for Secondary Annuitant, Surrender, Surrender Value, and UIN. 



In [14]:
from llama_index.core import get_response_synthesizer
from llama_index.llms.gemini import Gemini
llm_gemini=Gemini(api_key=userdata.get('gemini_api_key'),model_name="models/gemini-1.5-flash")
response_synthesizer_gemini = get_response_synthesizer(response_mode="tree_summarize", llm=llm_gemini)
query_engine_gemini = index_gemini.as_query_engine(llm=llm_gemini,response_synthesizer=response_synthesizer_gemini)
response_gemini = query_engine_gemini.query("List down all the definitions. All 20 of them?")
print(response_gemini)

I can provide you with definitions from the provided context, but there are only 4 definitions, not 20. 

* **Secondary Annuitant:** The person who receives annuity payments after the death of the primary annuitant.
* **Surrender:** The complete withdrawal or termination of the entire policy.
* **Surrender Value:** The amount payable upon surrender of the policy, if any.
* **UIN:** The Unique Identification Number assigned to the plan by the IRDAI. 



In [15]:
nodes = vector_retriever_jina.retrieve("What are all the definitions?")
print(nodes[0].get_content(), '\n\n\n\n\n *******', nodes[1].get_content(), '\n\n\n\n\n\n\n\ ****', nodes[2].get_content())

Note: In case of dispute in respect of interpretation of these terms and conditions mentioned in this document the English version shall stand valid.

# YOU ARE REQUESTED TO EXAMINE THIS POLICY, AND IF ANY MISTAKE BE FOUND THEREIN, RETURN IT IMMEDIATELY FOR CORRECTION.

# LIC’s New Jeevan Shanti (UIN: 512N338V05)

Page 15 of 21 




 ******* # PART – B: DEFINITIONS

The definitions of terms/words used in the Policy Document are as under:

1. Age is the age last birthday of the Annuitant(s) on the date of commencement of the policy.
2. Annuitant(s) being person(s) on whose life this policy has been taken and who become entitled to receive the annuity benefits as stated in Policy Schedule.
3. Annuity means a specified amount payable under this policy at specified regular intervals as mentioned in the schedule and payable as per the specification under the option chosen by the annuitant as evidenced in the schedule of the policy. Annuity option once chosen cannot be altered.
4. Appointee 

The definitions provided are: Secondary Annuitant, Surrender, Surrender Value, and UIN. 



In [21]:
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    RetrieverEvaluator,
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset)
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import DatasetGenerator
os.environ['OPENAI_API_KEY']=userdata.get('ChatGPT')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [22]:
data_generator = DatasetGenerator.from_documents(documents)
eval_dataset = data_generator.generate_dataset_from_nodes(num = 4)

/usr/local/lib/python3.10/dist-packages/llama_index/core/evaluation/dataset_generation.py:200: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/usr/local/lib/python3.10/dist-packages/llama_index/core/evaluation/dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [24]:
eval_questions = [example[0] for example in eval_dataset.qr_pairs]
eval_answers = [example[1] for example in eval_dataset.qr_pairs]

In [25]:
print(eval_questions)
print(eval_answers)

['What is the registration number of the Life Insurance Corporation of India?', "What type of plan is LIC's New Jeevan Shanti?", 'What is the Free Look Period mentioned in the policy document?', 'How long is the period within which a policyholder can return the policy if they disagree with the terms and conditions?']
['The registration number of the Life Insurance Corporation of India is 512.', "LIC's New Jeevan Shanti is a Non-Linked, Non-Participating, Individual, Single Premium, Deferred Annuity Plan.", 'The Free Look Period mentioned in the policy document allows the policyholder to review the terms and conditions of the policy. If the policyholder disagrees with any of the terms and conditions, they can return the policy within 30 days from the date of receipt of the electronic or physical mode of the policy document. Upon returning the policy, the premium amount deposited will be refunded after deducting charges for stamp duty and Annuity paid (if any).', 'The period within which

In [26]:
!pip install llama-index-llms-mistralai
from llama_index.llms.mistralai import MistralAI

In [27]:
# Fix GPT-3.5-TURBO LLM for generating response
llm_gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# Fix GPT-4o LLM for evaluation
llm_gpt4 = OpenAI(temperature=0, model="gpt-4o")

llm_gemini=Gemini(api_key=userdata.get('gemini_api_key'),model_name="models/gemini-1.5-flash")

llm_mistral= MistralAI(api_key=userdata.get('mistral'))


In [28]:
index_chatgpt = VectorStoreIndex.from_documents(
    documents, llm=OpenAI(temperature=0, model="gpt-4o")
)


#index_jina = VectorStoreIndex.from_documents(documents, embed_model=embed_model_jina, transformations=[token_text_splitter])

#index_gemini = VectorStoreIndex.from_documents(documents, embed_model=embed_model_gemini, transformations=[token_text_splitter])

#index_bgesmall = VectorStoreIndex.from_documents(documents, embed_model=embed_model_bgesmall, transformations=[token_text_splitter])

#index_openai = VectorStoreIndex.from_documents(documents, embed_model=embed_model_openai, transformations=[token_text_splitter])


In [29]:
query_engine_chatgpt = index_chatgpt.as_query_engine()
query_engine_jina = index_jina.as_query_engine()
query_engine_gemini = index_gemini.as_query_engine()
query_engine_bgesmall = index_bgesmall.as_query_engine()
query_engine_openai = index_openai.as_query_engine()

In [30]:
retriever_chatgpt = index_chatgpt.as_retriever(similarity_top_k=3)
#nodes_chatgpt = retriever_chatgpt.retrieve(eval_query)

retriever_jina = index_jina.as_retriever(similarity_top_k=3)
#nodes_jina = retriever_jina.retrieve(eval_query)

retriever_gemini = index_gemini.as_retriever(similarity_top_k=3)
#nodes_gemini = retriever_gemini.retrieve(eval_query)

retriever_bgesmall = index_bgesmall.as_retriever(similarity_top_k=3)
#nodes_bgesmall = retriever_bgesmall.retrieve(eval_query)

retriever_openai = index_openai.as_retriever(similarity_top_k=3)
#nodes_openai = retriever_openai.retrieve(eval_query)



In [31]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=llm_mistral)
relevancy_evaluator = RelevancyEvaluator(llm=llm_mistral)
correctness_evaluator = CorrectnessEvaluator(llm=llm_mistral)

In [33]:
import asyncio
from llama_index.core.evaluation import BatchEvalRunner

runner = BatchEvalRunner(
    {
     "faithfulness": faithfulness_evaluator,
     "relevancy": relevancy_evaluator,
     "correctness": correctness_evaluator
     },
    workers=1,
)

eval_results = await runner.aevaluate_queries(
    query_engine_openai, queries=eval_questions, reference = eval_answers
)
await asyncio.sleep(1)  # Delay of 1 second to reduce the frequency of requests


In [34]:
def get_eval_results(key, eval_results):
    results = eval_results[key]
    correct = 0
    for result in results:
        if result.passing:
            correct += 1
    score = correct / len(results)
    print(f"{key} Score: {score}")
    return score

In [35]:
_ = get_eval_results("relevancy", eval_results)

relevancy Score: 1.0


In [36]:
_ = get_eval_results("correctness", eval_results)

correctness Score: 1.0


In [37]:
_ = get_eval_results("faithfulness", eval_results)

faithfulness Score: 1.0


In [40]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=llm_gpt35)
relevancy_evaluator = RelevancyEvaluator(llm=llm_gpt35)
correctness_evaluator = CorrectnessEvaluator(llm=llm_gpt35)

In [41]:
import asyncio
from llama_index.core.evaluation import BatchEvalRunner

runner = BatchEvalRunner(
    {
     "faithfulness": faithfulness_evaluator,
     "relevancy": relevancy_evaluator,
     "correctness": correctness_evaluator
     },
    workers=1,
)

eval_results = await runner.aevaluate_queries(
    query_engine_openai, queries=eval_questions, reference = eval_answers
)
await asyncio.sleep(1)  # Delay of 1 second to reduce the frequency of requests


In [42]:
_ = get_eval_results("faithfulness", eval_results)

faithfulness Score: 1.0


In [43]:
_ = get_eval_results("correctness", eval_results)

correctness Score: 0.75


In [44]:
_ = get_eval_results("relevancy", eval_results)

relevancy Score: 1.0


In [45]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=llm_gemini)
relevancy_evaluator = RelevancyEvaluator(llm=llm_gemini)
correctness_evaluator = CorrectnessEvaluator(llm=llm_gemini)

In [46]:
import asyncio
from llama_index.core.evaluation import BatchEvalRunner

runner = BatchEvalRunner(
    {
     "faithfulness": faithfulness_evaluator,
     "relevancy": relevancy_evaluator,
     "correctness": correctness_evaluator
     },
    workers=1,
)

eval_results = await runner.aevaluate_queries(
    query_engine_openai, queries=eval_questions, reference = eval_answers
)
await asyncio.sleep(1)  # Delay of 1 second to reduce the frequency of requests


In [47]:
_ = get_eval_results("faithfulness", eval_results)


faithfulness Score: 1.0


In [48]:
_ = get_eval_results("relevancy", eval_results)

relevancy Score: 1.0


In [49]:
_ = get_eval_results("correctness", eval_results)

correctness Score: 1.0
